<a href="https://colab.research.google.com/github/Muhammad-Lutfi-2002/Data_analytics/blob/main/Survey_Participation_Analytics_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading dan Initial Cleaning
def load_and_clean_data():
    # Load datasets
    users_df = pd.read_csv('users.csv')
    surveys_df = pd.read_csv('surveys.csv')
    participations_df = pd.read_csv('survey_participations.csv')
    payments_df = pd.read_csv('payments.csv')
    # Convert date columns
    date_columns = {
        'users_df': ['created_at'],
        'surveys_df': ['created_at', 'end_at'],
        'participations_df': ['response_date'],
        'payments_df': ['payment_date']
    }

    for df_name, columns in date_columns.items():
        for col in columns:
            locals()[df_name][col] = pd.to_datetime(locals()[df_name][col])

    return users_df, surveys_df, participations_df, payments_df

In [ ]:
# 2. Data Cleaning Functions
def clean_users_data(df):
    # Copy dataframe to avoid modifications to original
    df_clean = df.copy()
    # Handle missing values
    df_clean['gender'] = df_clean['gender'].fillna('Not Specified')
    df_clean['education'] = df_clean['education'].fillna('Not Specified')
    df_clean['occupation'] = df_clean['occupation'].fillna('Not Specified')
    df_clean['city'] = df_clean['city'].fillna('Unknown')
    # Standardize text columns
    text_columns = ['gender', 'education', 'occupation', 'city']
    for col in text_columns:
        df_clean[col] = df_clean[col].str.strip().str.title()

    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['user_id'])

    return df_clean

In [ ]:
def clean_surveys_data(df):
    df_clean = df.copy()

    # Handle missing values
    df_clean['reward'] = df_clean['reward'].fillna(df_clean['reward'].mean())
    df_clean['participant_needed'] = df_clean['participant_needed'].fillna(0)
    df_clean['status'] = df_clean['status'].fillna('Unknown')

    # Remove surveys with invalid dates
    df_clean = df_clean[df_clean['end_at'] >= df_clean['created_at']]

    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['survey_id'])

    return df_clean

In [ ]:
def clean_participations_data(df):
    df_clean = df.copy()

    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['participation_id'])

    # Sort by response date
    df_clean = df_clean.sort_values('response_date')

    return df_clean

In [ ]:
def clean_payments_data(df):
    df_clean = df.copy()

    # Remove negative or zero amounts
    df_clean = df_clean[df_clean['amount'] > 0]

    # Remove duplicates
    df_clean = df_clean.drop_duplicates(subset=['payment_id'])

    # Sort by payment date
    df_clean = df_clean.sort_values('payment_date')

    return df_clean

In [ ]:
# 3. Data Validation Functions
def validate_data_integrity(users_df, surveys_df, participations_df, payments_df):
    validation_results = {
        'missing_users': [],
        'missing_surveys': [],
        'invalid_payments': []
    }
    # Check if all users in participations exist in users table
    participating_users = participations_df['user_id'].unique()
    existing_users = users_df['user_id'].unique()
    validation_results['missing_users'] = [u for u in participating_users if u not in existing_users]
    # Check if all surveys in participations exist in surveys table
    participating_surveys = participations_df['survey_id'].unique()
    existing_surveys = surveys_df['survey_id'].unique()
    validation_results['missing_surveys'] = [s for s in participating_surveys if s not in existing_surveys]

    # Check payment amounts against survey rewards
    merged_payments = payments_df.merge(surveys_df[['survey_id', 'reward']], on='survey_id', how='left')
    validation_results['invalid_payments'] = merged_payments[
        merged_payments['amount'] > merged_payments['reward'] * 1.1
    ]['payment_id'].tolist()

    return validation_results



In [ ]:
# 4. Create Clean Dataset
def create_clean_dataset():
    # Load raw data
    users_df, surveys_df, participations_df, payments_df = load_and_clean_data()
    # Clean individual datasets
    users_clean = clean_users_data(users_df)
    surveys_clean = clean_surveys_data(surveys_df)
    participations_clean = clean_participations_data(participations_df)
    payments_clean = clean_payments_data(payments_df)
    # Validate data integrity
    validation_results = validate_data_integrity(
        users_clean,
        surveys_clean,
        participations_clean,
        payments_clean
    )
    # Remove invalid records based on validation
    if validation_results['missing_users']:
        participations_clean = participations_clean[
            ~participations_clean['user_id'].isin(validation_results['missing_users'])
        ]

    if validation_results['missing_surveys']:
        participations_clean = participations_clean[
            ~participations_clean['survey_id'].isin(validation_results['missing_surveys'])
        ]

    if validation_results['invalid_payments']:
        payments_clean = payments_clean[
            ~payments_clean['payment_id'].isin(validation_results['invalid_payments'])
        ]

    # Save clean datasets
    users_clean.to_csv('clean_users.csv', index=False)
    surveys_clean.to_csv('clean_surveys.csv', index=False)
    participations_clean.to_csv('clean_participations.csv', index=False)
    payments_clean.to_csv('clean_payments.csv', index=False)

    return users_clean, surveys_clean, participations_clean, payments_clean

In [ ]:
# 5. Create Analysis Ready Views
def create_analysis_views(users_df, surveys_df, participations_df, payments_df):
    # User Demographics View
    user_demographics = users_df.groupby(['city', 'gender', 'education']).size().reset_index(name='user_count')
    # Survey Performance View
    survey_performance = surveys_df.merge(
        participations_df.groupby('survey_id').size().reset_index(name='actual_participants'),
        on='survey_id',
        how='left'
    )

    # User Participation View
    user_participation = participations_df.merge(
        users_df[['user_id', 'city', 'gender', 'education']],
        on='user_id',
        how='left'
    )

    # Payment Analysis View
    payment_analysis = payments_df.merge(
        surveys_df[['survey_id', 'title', 'reward']],
        on='survey_id',
        how='left'
    ).merge(
        users_df[['user_id', 'city', 'gender']],
        on='user_id',
        how='left'
    )

    return user_demographics, survey_performance, user_participation, payment_analysis

In [ ]:
# Execute the cleaning process
if __name__ == "__main__":
    # Get clean datasets
    users_clean, surveys_clean, participations_clean, payments_clean = create_clean_dataset()

    # Create analysis views
    user_demographics, survey_performance, user_participation, payment_analysis = create_analysis_views(
        users_clean,
        surveys_clean,
        participations_clean,
        payments_clean
    )

    # Save analysis views
    user_demographics.to_csv('analysis_user_demographics.csv', index=False)
    survey_performance.to_csv('analysis_survey_performance.csv', index=False)
    user_participation.to_csv('analysis_user_participation.csv', index=False)
    payment_analysis.to_csv('analysis_payment.csv', index=False)